# Guided Project on Chinook database

This is my execution of the DataQuest Guided Project. This project is guided insofar as the DataQuest platform provides a set of prompts to be answered using SQL to analyze the Chinook database.

The Chinook SQLite database file represents a digital media store with information on track sales, customer ids, etc. This is a sample database which uses partially real iTunes data, but was created for SQL practice.

### Connect Chinook database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Explore

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Which genres sell the most tracks in the U.S.?

In [3]:
%%sql

WITH tracks_sold_usa AS
                       (
                        SELECT il.* 
                        FROM invoice_line il
                        INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                        INNER JOIN customer c ON i.customer_id = c.customer_id
                        WHERE c.country = "USA"
                       )
    
SELECT g.name genre, 
       COUNT(tsu.invoice_line_id) sold_by_genre,
       ROUND(CAST(COUNT(tsu.invoice_line_id) AS FLOAT) / 
        (
        SELECT COUNT(*) from tracks_sold_usa
        ), 3) percentage
FROM tracks_sold_usa tsu
INNER JOIN track t ON t.track_id = tsu.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1 
ORDER BY 2 DESC;            


Done.


genre,sold_by_genre,percentage
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


### For each country, calculate:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [4]:
%%sql

WITH joined_table AS 
                   (
                    SELECT 
                      CASE WHEN (
                            SELECT COUNT(*)
                            FROM customer
                            WHERE country = c.country
                            ) = 1 THEN "Other"
                           ELSE c.country
                           END 
                           AS country, 
                       c.customer_id, 
                       i.invoice_id, 
                       i.total
                    FROM customer c
                    INNER JOIN invoice i ON i.customer_id = c.customer_id
                   )

SELECT country, 
       COUNT(DISTINCT customer_id) n_customers, 
       ROUND(SUM(total),3) total_sales,
       ROUND(SUM(total) / COUNT(DISTINCT customer_id), 3) avg_sales_per_customer,
       ROUND(SUM(total) / (
                      SELECT COUNT(j.invoice_id)
                      FROM joined_table
                      GROUP BY country
                      ),3) avg_order_value
FROM joined_table j
GROUP BY 1 ORDER BY country = "Other" ASC, 3 DESC;

Done.


country,n_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.038,7.943
Canada,8,535.59,66.949,7.047
Brazil,5,427.68,85.536,7.011
France,5,389.07,77.814,7.781
Germany,4,334.62,83.655,8.161
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.52,81.84,8.769
Portugal,2,185.13,92.565,6.384
India,2,183.15,91.575,8.721
Other,15,1094.94,72.996,7.449


### What percentage of purchases are albums v. individual tracks?
The digital media store allows to purchase either a full album or on or more individual tracks, but not a mix of both. 

_There is an edge case which prevents a completely accurate analysis: customers can manually select all individual tracks from one album and then add more individual tracks. If this occurs, the purchase will be a mix of full albums and individual tracks, but in the analysis below it will be included in the percentage of individual tracks._

In [14]:
%%sql

WITH a_track_per_invoice AS 
                (
                SELECT invoice_id, MAX(track_id) track_id
                FROM invoice_line il
                GROUP BY invoice_id
                ),
    
album_vs_individual AS
    (
    SELECT  a_track_per_invoice.*,
        CASE 
           WHEN (
                 SELECT il2.track_id FROM invoice_line il2
                 WHERE il2.invoice_id = a_track_per_invoice.invoice_id
    
                 EXCEPT
        
                 SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT tr.album_id FROM track tr
                                WHERE tr.track_id = a_track_per_invoice.track_id)
                )
        IS NULL
        
        AND (
            SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT tr.album_id FROM track tr
                                WHERE tr.track_id = a_track_per_invoice.track_id)
            EXCEPT
            
            SELECT il2.track_id FROM invoice_line il2
                 WHERE il2.invoice_id = a_track_per_invoice.invoice_id
            )
        
        THEN "yes"
        ELSE "no"
        END AS album_purchase
FROM a_track_per_invoice
    )

        
SELECT album_purchase, 
       COUNT(invoice_id) n_invoices, 
       ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT * FROM invoice), 3) percent_invoices
FROM album_vs_individual
GROUP BY album_purchase

(sqlite3.OperationalError) near "FROM": syntax error
[SQL: WITH a_track_per_invoice AS 
                (
                SELECT invoice_id, MAX(track_id) track_id
                FROM invoice_line il
                GROUP BY invoice_id
                ),
    
album_vs_individual AS
    (
    SELECT  a_track_per_invoice.*,
        CASE 
           WHEN (
                 SELECT il2.track_id FROM invoice_line il2
                 WHERE il2.invoice_id = a_track_per_invoice.invoice_id
    
                 EXCEPT
        
                 SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT tr.album_id FROM track tr
                                WHERE tr.track_id = a_track_per_invoice.track_id)
                )
        IS NULL
        
        AND (
            SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT tr.album_id FROM track tr
                                WHERE tr.track_id = a_track_per_invoice.track_id)
            EXCEPT
    

In [29]:
%%sql

WITH a_track_per_invoice AS 
                (
                SELECT invoice_id, MAX(track_id) track_id
                FROM invoice_line il
                GROUP BY invoice_id
                ),
    
album_vs_individual AS
    (
    SELECT  a_track_per_invoice.*,
        CASE 
           WHEN (
                 SELECT il2.track_id FROM invoice_line il2
                 WHERE il2.invoice_id = a_track_per_invoice.invoice_id
    
                 EXCEPT
        
                 SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT tr.album_id FROM track tr
                                WHERE tr.track_id = a_track_per_invoice.track_id)
                )
        IS NULL
        
        AND (
            SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT tr.album_id FROM track tr
                                WHERE tr.track_id = a_track_per_invoice.track_id)
            EXCEPT
            
            SELECT il2.track_id FROM invoice_line il2
                 WHERE il2.invoice_id = a_track_per_invoice.invoice_id
            )
        
        IS NULL
        
        THEN "Individual_tracks"
        ELSE "Album"
        END AS album_purchase
FROM a_track_per_invoice
    )

SELECT album_purchase, 
       COUNT(invoice_id) n_of_invoices,
       ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice), 3) percentage
FROM album_vs_individual
GROUP BY album_purchase

Done.


album_purchase,n_of_invoices,percentage
Album,500,0.814
Individual_tracks,114,0.186


### Which artists are most used in most playlists?

In [7]:
%%sql

WITH new_table AS 
                (
                 SELECT * 
                 FROM artist a
                 LEFT JOIN album al ON al.artist_id = a.artist_id
                 LEFT JOIN track t ON t.album_id = al.album_id
                 LEFT JOIN playlist_track p ON p.track_id = t.track_id
                )
    
SELECT name, COUNT(DISTINCT playlist_id) n_playlists
FROM new_table
GROUP BY 1 ORDER BY 2 DESC
LIMIT 10;

Done.


name,n_playlists
Eugene Ormandy,7
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Berliner Philharmoniker & Herbert Von Karajan,6
English Concert & Trevor Pinnock,6
The King's Singers,6
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5
Adrian Leaper & Doreen de Feis,5
Alberto Turco & Nova Schola Gregoriana,5
Antal Doráti & London Symphony Orchestra,5
Barry Wordsworth & BBC Concert Orchestra,5


### How does the range of tracks in the store compare to their popularity?
I define "range" as genre and "popularity" as tracks sold. I use and modify the code from line 4.

In [8]:
%%sql

WITH tracks_sold AS
                    (
                    SELECT il.* 
                    FROM invoice_line il
                    INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                    INNER JOIN customer c ON i.customer_id = c.customer_id
                    ),
    
tracks_sold_by_genre AS 
                      (
                      SELECT g.name genre, 
                      COUNT(ts.invoice_line_id) sold_by_genre,
                      ROUND(CAST(COUNT(ts.invoice_line_id) AS FLOAT) / 
                      (
                      SELECT COUNT(*) from tracks_sold
                      ), 3) percentage_sold
                      FROM tracks_sold ts
                      INNER JOIN track t ON t.track_id = ts.track_id
                      INNER JOIN genre g ON g.genre_id = t.genre_id
                      GROUP BY 1 
                      ORDER BY 2 DESC
                      )

SELECT g.name genre, 
       ROUND(CAST(COUNT(track_id) AS FLOAT) / (
        SELECT COUNT(*) FROM track
       ), 3) percentage_in_store,
       percentage_sold 
FROM track t
INNER JOIN genre g ON g.genre_id = t.genre_id
INNER JOIN tracks_sold_by_genre tt ON tt.genre = g.name
GROUP BY 1 ORDER BY 2 DESC;

Done.


genre,percentage_in_store,percentage_sold
Rock,0.37,0.554
Latin,0.165,0.035
Metal,0.107,0.13
Alternative & Punk,0.095,0.103
Jazz,0.037,0.025
TV Shows,0.027,0.0
Blues,0.023,0.026
Classical,0.021,0.01
Drama,0.018,0.0
R&B/Soul,0.017,0.033
